## 5 - É possível identificar casos inconsistentes utilizando classificação de NCM?

- Target: CÓDIGO NCM/SH

- Texto: DESCRIÇÃO DO PRODUTO/SERVIÇO

### Importação de Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import datetime
import os
from sklearn.metrics import classification_report

import tensorflow_addons as tfa
import keras_tuner as kt
from tensorflow import keras
import tensorflow as tf

from classes import Preprocessing, Model, Lstm

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

2023-04-04 11:00:23.993249: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-04 11:00:24.039989: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-04 11:00:24.040631: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-04 11:00:24.879964: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mayara/Documentos/tcc/nfe-analysis/Nfe-Analysis/.venv/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions ar

### Leitura de Dados

In [2]:
raw_df = pd.read_csv('data/processed_nfe/nfe_100000.csv')

### Pré-processamento

Tipagem

In [3]:
df = Preprocessing.define_types(raw_df)
df = Preprocessing.filter_event_authorized(df)

Definição de colunas

In [4]:
df['CAPÍTULO NCM'] = df['CÓDIGO NCM/SH'].astype(str).str[0] + df['CÓDIGO NCM/SH'].astype(str).str[1]
df = df[['DESCRIÇÃO DO PRODUTO/SERVIÇO','CAPÍTULO NCM']]
df.rename(columns={'DESCRIÇÃO DO PRODUTO/SERVIÇO':'DESCRICAO'},inplace=True)

df

/tmp/ipykernel_16657/4265368263.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CAPÍTULO NCM'] = df['CÓDIGO NCM/SH'].astype(str).str[0] + df['CÓDIGO NCM/SH'].astype(str).str[1]


,DESCRICAO,CAPÍTULO NCM
0,CEBOLA KG,70
2,1020-1200 #FRESA INICIAL ACCOLADE TMFZ,90
3,NAN COMFOR 2 400G - NESTLE,19
4,OLIGONUCLEOTIDEOS - IDT,29
5,CARTUCHO DE TONER COMPATÍVEL SAMSUNG CTL 406Y ...,84
...,...,...
99994,TE RED SOLD - 75 X 50MM,39
99995,MANGUEIRA PESC GAS. HUSQ UNIV,39
99996,PARAFUSO PEDICULAR POL. PEDIMAX II 06.2x55mm,90
99997,PARAFUSO DE BLOQUEIO PBA 03.5x38mm,90


In [5]:
df['DESCRICAO'].to_csv('raw_description.csv')

In [39]:
list(filter(lambda a: a != 2, [1,2,3,4,2]))

TypeError: 'list' object is not callable

In [35]:
list = [['cefalexina', 'mg', 'LÍQUIDO', 'mg', 'ml', 'LÍQUIDO', 'po', 'susp', 'fr', 'gen', 'abl'],
        ['adesivo', 'uso', 'gera', 'LÍQUIDO', 'veda', 'calha'],
        ['adesivo', 'uso', 'gera', 'veda', 'calha'],
        ['LÍQUIDO', 'adesivo', 'LÍQUIDO', 'uso', 'gera', 'veda', 'calha','LÍQUIDO']]

for l in list:
    if 'LÍQUIDO' in l:
        l = filter(lambda a: a != 2, l)
        l.append('LÍQUIDO')
        # liq_count = l.count('LÍQUIDO')
        # for c in range(0,liq_count-1):
        #     # print(l)
        #     l.remove('LÍQUIDO')

print(list)

AttributeError: 'filter' object has no attribute 'list'

In [18]:
import re
re.sub(r'[\d]', ' NUMERO ', '/ 12.5K P/ M3145/P3045DN/P3145 200.220,00')

'/  NUMERO  NUMERO . NUMERO K P/ M NUMERO  NUMERO  NUMERO  NUMERO /P NUMERO  NUMERO  NUMERO  NUMERO DN/P NUMERO  NUMERO  NUMERO  NUMERO   NUMERO  NUMERO  NUMERO . NUMERO  NUMERO  NUMERO , NUMERO  NUMERO '

In [21]:
import re
re.sub(r'[\d.]*[\d]+[,\d]*', '', '/ 12.5K P/ M3145/P3045DN/P3145 200.220,00')

'/ K P/ M/PDN/P '

In [6]:
ncms_list = df['CAPÍTULO NCM'].value_counts()[0:50].index.tolist()
df = df[df['CAPÍTULO NCM'].isin(ncms_list)]

Aplica pré-processamento no texto da 'DESCRICAO'

In [7]:
teste, corpus_teste = Preprocessing.apply_preprocessing(df)

teste

,DESCRICAO,CAPÍTULO NCM
0,"[cebola, kg]",70
2,"[fresa, inicia, LÍQUIDO, accolade, tmfz]",90
3,"[nan, comfor, g, nestle]",19
4,"[oligonucleotideos, idt]",29
5,"[cartucho, toner, compatíve, LÍQUIDO, samsung,...",84
...,...,...
99994,"[red, sold, mm]",39
99995,"[mangueira, pesc, gas, husq, univ]",39
99996,"[parafuso, pedicular, pol, pedimax, ii, xmm]",90
99997,"[parafuso, bloqueio, pba, xmm]",90


In [8]:
teste['DESCRICAO'].to_csv('description.csv')

In [11]:
df, corpus_desc = Preprocessing.apply_preprocessing(df)

df.head()

,DESCRICAO,CAPÍTULO NCM
0,"[cebola, kg]",70
2,"[fresa, inicial, accolade, tmfz]",90
3,"[nan, comfor, QUANTITY, nestle]",19
4,"[oligonucleotideos, idt]",29
5,"[cartucho, toner, compatível, samsung, ctl, y,...",84


In [7]:
df_train, df_val, df_test = Preprocessing().split_dataset(df,['DESCRICAO'],'CAPÍTULO NCM')

train: 70%
val: 10%
test: 20%


In [8]:
len(df_train['CAPÍTULO NCM'].unique())

50

In [9]:
len(df_val['CAPÍTULO NCM'].unique())

50

In [10]:
mean_sequence_length, max_sequence_length = Preprocessing.get_sequences_details(df_train)

print(f'Mean sequence length: {mean_sequence_length}')
print(f'Max sequence length: {max_sequence_length}')

Mean sequence length: 4.781898377414373
Max sequence length: 25


In [11]:
MAX_SEQUENCE_LENGTH = max_sequence_length
NUM_LABELS = len(ncms_list)

VOCAB_SIZE, X_train_padded, X_val_padded, X_test_padded = Preprocessing.adapt_X_for_input_layer(df_train['DESCRICAO'].astype(str), df_val['DESCRICAO'].astype(str), df_test['DESCRICAO'].astype(str), MAX_SEQUENCE_LENGTH)

print('Número de labels: ', NUM_LABELS)
print('Training features shape:', X_train_padded.shape)
print('Validation features shape:', X_val_padded.shape)
print('Test features shape:', X_test_padded.shape)

X_train_padded

Número de labels:  50
Training features shape: (62801, 25)
Validation features shape: (8972, 25)
Test features shape: (17944, 25)


array([[    0,     0,     0, ...,     4,     1,    43],
       [    0,     0,     0, ...,   906,   404,  2521],
       [    0,     0,     0, ...,     1,   567, 12366],
       ...,
       [    0,     0,     0, ...,   166,  2693,     8],
       [    0,     0,     0, ...,     0,   167,   896],
       [    0,     0,     0, ...,     0,    95,   447]], dtype=int32)

In [12]:
y_train_cat, y_val_cat, y_test_cat = Preprocessing.adapt_y_for_input_layer(df_train['CAPÍTULO NCM'], df_val['CAPÍTULO NCM'], df_test['CAPÍTULO NCM'])

print('Training features shape:', y_train_cat.shape)
print('Validation features shape:', y_val_cat.shape)
print('Test features shape:', y_test_cat.shape)

Training features shape: (62801, 50)
Validation features shape: (8972, 50)
Test features shape: (17944, 50)


In [13]:
METRICS = [
      'accuracy'
]

In [14]:
tuner = kt.RandomSearch(
    hypermodel=Lstm(VOCAB_SIZE, MAX_SEQUENCE_LENGTH, NUM_LABELS, METRICS),
    objective='accuracy',
    max_trials=6,
    executions_per_trial=2,
    overwrite=True,
    directory="models/hyperparameters_search",
    project_name="lstm",
    seed=SEED
)

print(tuner.search_space_summary())

Search space summary
Default search space size: 5
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 96, 'step': 8, 'sampling': 'linear'}
activation1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.2, 'step': 0.1, 'sampling': 'linear'}
activation2 (Choice)
{'default': 'softmax', 'conditions': [], 'values': ['softmax'], 'ordered': False}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.005, 'step': None, 'sampling': 'log'}
None


2023-04-03 13:36:47.018846: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [15]:
logdir = os.path.join("models/logs/lstm/", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)
earlystopping_callback = keras.callbacks.EarlyStopping('val_loss', mode='min', verbose=1, patience=5)

callbacks_list = [earlystopping_callback, tensorboard_callback]

In [16]:
tuner.search(X_train_padded, y_train_cat,
             validation_data=(X_val_padded, y_val_cat),
             callbacks=callbacks_list)

Trial 6 Complete [00h 15m 57s]
accuracy: 0.7907994985580444

Best accuracy So Far: 0.8796914219856262
Total elapsed time: 01h 43m 32s
INFO:tensorflow:Oracle triggered exit


In [17]:
best_model = tuner.get_best_models()[0]
Model.save(best_model, 'saved_models/lstm')

best_hps = tuner.get_best_hyperparameters()[0]
print(best_hps.values)

{'units': 72, 'activation1': 'relu', 'rate': 0.1, 'activation2': 'softmax', 'learning_rate': 0.0032295411136862955, 'batch_size': 8, 'epochs': 4}


In [18]:
hypermodel = Model.recover('saved_models/lstm')

EPOCHS = 4
BATCH_SIZE = 8
LEARNING_RATE = 0.0032295411136862955
LOSS = 'categorical_crossentropy'

hypermodel.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                    loss=LOSS, 
                    metrics=METRICS) 

In [19]:
eval_result_val = hypermodel.evaluate(X_val_padded, y_val_cat)

y_probabilities_val = hypermodel.predict(X_val_padded)
y_pred_val = np.argmax(y_probabilities_val, axis=1)
y_val = np.argmax(y_val_cat, axis=1)

print('\nValidation')
print(classification_report(y_val, y_pred_val, zero_division=True))

2023-04-03 15:20:27.804379: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype uint8 and shape [8972,50]
	 [[{{node Placeholder/_11}}]]
2023-04-03 15:20:27.805096: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype uint8 and shape [8972,50]
	 [[{{node Placeholder/_11}}]]


  1/281 [..............................] - ETA: 46s

2023-04-03 15:20:28.885719: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int32 and shape [8972,25]
	 [[{{node Placeholder/_10}}]]
2023-04-03 15:20:28.885950: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int32 and shape [8972,25]
	 [[{{node Placeholder/_10}}]]


281/281 [==============================] - 1s 2ms/step

Validation
              precision    recall  f1-score   support

           0       0.65      0.59      0.62        29
           1       0.77      0.77      0.77        57
           2       0.83      0.71      0.77        21
           3       0.83      0.85      0.84        73
           4       0.96      0.62      0.75        39
           5       0.59      0.71      0.65        14
           6       0.80      0.86      0.83       194
           7       0.79      0.70      0.74       201
           8       0.71      0.78      0.74       193
           9       0.89      0.82      0.85       125
          10       0.75      0.50      0.60        18
          11       0.85      0.84      0.85        49
          12       0.91      0.95      0.93       288
          13       0.87      0.84      0.85       112
          14       0.52      0.55      0.54        40
          15       0.80      0.85      0.82       321
          16  

In [20]:
eval_result_test = hypermodel.evaluate(X_test_padded, y_test_cat)

y_probabilities_test = hypermodel.predict(X_test_padded)
y_pred_test = np.argmax(y_probabilities_test, axis=1)
y_test = np.argmax(y_test_cat, axis=1)

print('\Test')
print(classification_report(y_test, y_pred_test, zero_division=True))

 63/561 [==>...........................] - ETA: 1s - loss: 0.9148 - accuracy: 0.7932

2023-04-03 15:20:31.775997: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype uint8 and shape [17944,50]
	 [[{{node Placeholder/_11}}]]
2023-04-03 15:20:31.776308: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype uint8 and shape [17944,50]
	 [[{{node Placeholder/_11}}]]


 39/561 [=>............................] - ETA: 2s

2023-04-03 15:20:33.465658: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int32 and shape [17944,25]
	 [[{{node Placeholder/_10}}]]
2023-04-03 15:20:33.466089: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int32 and shape [17944,25]
	 [[{{node Placeholder/_10}}]]


561/561 [==============================] - 3s 5ms/step
\Test
              precision    recall  f1-score   support

           0       0.69      0.58      0.63        64
           1       0.85      0.76      0.80       109
           2       0.90      0.80      0.85        65
           3       0.85      0.83      0.84       164
           4       0.87      0.72      0.79        57
           5       0.72      0.79      0.75        33
           6       0.83      0.87      0.85       348
           7       0.74      0.68      0.71       367
           8       0.72      0.80      0.76       355
           9       0.80      0.86      0.83       216
          10       1.00      0.36      0.53        28
          11       0.81      0.74      0.77        78
          12       0.90      0.96      0.93       576
          13       0.77      0.75      0.76       181
          14       0.51      0.45      0.48        95
          15       0.82      0.88      0.85       762
          16       0